In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
path = r'd:\Documentos\Projects\Python\EMDR-Project\src\sensor\data\sensor_data_20250611_195404.csv'

In [5]:
# Cargar el archivo CSV en un DataFrame
df = pd.read_csv(path, 
                 encoding='utf-8',  # Si hay caracteres especiales
                 index_col=0)  # Si la primera columna es el índice

# Mostrar información básica del DataFrame
print("Información del DataFrame:")
print(df.info())
print("\nPrimeras 5 filas:")
print(df.head())
print("\nÚltimas 5 filas:")
print(df.tail())
print(f"\nForma del DataFrame: {df.shape}")
print(f"\nTiempo total:")
# Convertir milisegundos a segundos y minutos
milisegundos = 1924136

# Convertir a segundos
segundos_total = milisegundos / 1000
print(f"Total en segundos: {segundos_total}")

# Convertir a minutos y segundos
minutos = int(segundos_total // 60)
segundos = segundos_total % 60

print(f"Tiempo: {minutos} minutos y {segundos:.3f} segundos")

Información del DataFrame:
<class 'pandas.core.frame.DataFrame'>
Index: 240518 entries, 0 to 240517
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Timestamp_ms  240518 non-null  int64  
 1   EOG_raw       240518 non-null  int64  
 2   EOG_filtrado  240518 non-null  float64
 3   PPG_raw       240518 non-null  int64  
 4   PPG_filtrado  240518 non-null  float64
 5   BPM           240518 non-null  float64
dtypes: float64(3), int64(3)
memory usage: 12.8 MB
None

Primeras 5 filas:
    Timestamp_ms  EOG_raw  EOG_filtrado  PPG_raw  PPG_filtrado  BPM
ID                                                                 
0              0      611      2.119196   -12662    -10.210935  0.0
1              8    -2097      9.909152   -15130    -83.490696  0.0
2             16    -3140     -6.843396   -16798   -337.215300  0.0
3             24    -1581   -173.013615   -17906   -917.829791  0.0
4             32    -5720   -665.

In [6]:
import ipywidgets as widgets
from IPython.display import display

In [7]:
# Habilitar widgets interactivos
%matplotlib widget

In [ ]:
# Convertir timestamp a segundos relativos
df['segundos'] = (df['Timestamp_ms'] - df['Timestamp_ms'].iloc[0]) / 1000

# Calcular número total de ventanas de 30 segundos
duracion_total = df['segundos'].iloc[-1]
num_ventanas = int(duracion_total // 30) + 1

interactive(children=(IntSlider(value=0, description='Inicio (s):', max=1894, step=30, style=SliderStyle(descr…

<function __main__.plot_eog_section(inicio_segundos=0)>

In [9]:
def plot_eog_section(inicio_segundos=0):
    fig, ax = plt.subplots(figsize=(12, 6))
    
    # Filtrar datos para ventana de 30 segundos
    fin_segundos = inicio_segundos + 30
    mask = (df['segundos'] >= inicio_segundos) & (df['segundos'] <= fin_segundos)
    datos_ventana = df[mask]
    
    ax.plot(datos_ventana['segundos'], datos_ventana['PPG_raw'], 'b-', linewidth=0.8)
    ax.set_xlabel('Tiempo (segundos)')
    ax.set_ylabel('PPG Raw')
    ax.set_title(f'PPG Raw - Segundos {inicio_segundos} a {fin_segundos}')
    ax.grid(True, alpha=0.3)
    ax.set_xlim(inicio_segundos, fin_segundos)
    
    plt.tight_layout()
    plt.show()

In [10]:


# Crear slider interactivo
slider = widgets.IntSlider(
    value=0,
    min=0,
    max=int(duracion_total - 30),
    step=30,
    description='Inicio (s):',
    style={'description_width': 'initial'}
)

# Conectar slider con la función
widgets.interact(plot_eog_section, inicio_segundos=slider)

interactive(children=(IntSlider(value=0, description='Inicio (s):', max=1894, step=30, style=SliderStyle(descr…

<function __main__.plot_eog_section(inicio_segundos=0)>

 Bases de datos

In [1]:
import sys
import os

# Agregar el directorio raíz del proyecto al path
project_root = r'd:\Documentos\Projects\Python\EMDR-Project'
if project_root not in sys.path:
    sys.path.append(project_root)

# Importar el DatabaseManager
from src.database.database_manager import DatabaseManager
import pandas as pd

In [2]:
# Verificar conexión y mostrar estructura de las tablas
print("=== ESTRUCTURA DE LA BASE DE DATOS ===\n")

# Obtener todos los pacientes
pacientes = DatabaseManager.get_all_patients()
print(f"📋 PACIENTES ({len(pacientes)} registros):")
print("-" * 80)

if pacientes:
    df_pacientes = pd.DataFrame(pacientes)
    print(df_pacientes.to_string(index=False))
    print("\nColumnas disponibles:", list(df_pacientes.columns))
else:
    print("No hay pacientes registrados")

print("\n" + "="*80 + "\n")

=== ESTRUCTURA DE LA BASE DE DATOS ===

📋 PACIENTES (3 registros):
--------------------------------------------------------------------------------
 id apellido_paterno apellido_materno nombre fecha_nacimiento  celular      fecha_registro      comentarios
  1          Serrudo          Careaga   Eloy       2002-01-01 78337255 2025-06-13 06:15:01           Prueba
  2                x                x Carlos       2002-01-01 12345678 2025-06-13 06:15:01 Segundo paciente
  3                x                x    seb       2002-01-01 12345678 2025-06-13 06:15:01                x

Columnas disponibles: ['id', 'apellido_paterno', 'apellido_materno', 'nombre', 'fecha_nacimiento', 'celular', 'fecha_registro', 'comentarios']




In [ ]:
# Obtener todas las sesiones (necesitamos iterar por cada paciente)
print(f"🔬 SESIONES:")
print("-" * 80)

all_sessions = []

if pacientes:
    for paciente in pacientes:
        patient_id = paciente['id']
        patient_name = f"{paciente['apellido_paterno']} {paciente['apellido_materno']} {paciente['nombre']}"
        
        # Obtener sesiones del paciente
        sesiones = DatabaseManager.get_sessions_for_patient(patient_id)
        
        # Agregar información del paciente a cada sesión
        for sesion in sesiones:
            sesion['paciente_id'] = patient_id
            sesion['paciente_nombre'] = patient_name
            all_sessions.append(sesion)

if all_sessions:
    df_sesiones = pd.DataFrame(all_sessions)
    print(f"Total de sesiones: {len(all_sessions)}")
    print(df_sesiones.to_string(index=False))
    print("\nColumnas disponibles:", list(df_sesiones.columns))
else:
    print("No hay sesiones registradas")

print("\n" + "="*80 + "\n")

🔬 SESIONES:
--------------------------------------------------------------------------------
Total de sesiones: 3
 id               fecha                                                                   comentarios  paciente_id      paciente_nombre
  1 2025-05-26 16:41:51  Sesión actualizada el 2025-05-26 16:43:23. Datos almacenados: 5545 muestras.            1 Serrudo Careaga Eloy
  2 2025-05-28 19:16:09 Sesión actualizada el 2025-05-28 19:16:13. Datos almacenados: 18895 muestras.            2           x x Carlos
  3 2025-05-29 10:53:39 Sesión actualizada el 2025-05-29 10:53:43. Datos almacenados: 15646 muestras.            3              x x seb

Columnas disponibles: ['id', 'fecha', 'comentarios', 'paciente_id', 'paciente_nombre']




In [6]:
import numpy as np
import pickle
import zlib

In [7]:
def generar_timestamps_ms(num_muestras, frecuencia_hz=125):
    """
    Genera timestamps en milisegundos comenzando desde 0
    
    Args:
        num_muestras: Número total de muestras
        frecuencia_hz: Frecuencia de muestreo (125 Hz por defecto)
    
    Returns:
        numpy array con timestamps en milisegundos
    """
    # Calcular intervalo entre muestras en milisegundos
    intervalo_ms = 1000 / frecuencia_hz  # 1000ms / 125Hz = 8ms entre muestras
    
    # Generar array de timestamps comenzando en 0
    timestamps = np.arange(num_muestras) * intervalo_ms
    
    return timestamps.astype(np.int64)

def comprimir_datos(datos):
    """Comprime datos usando pickle + zlib"""
    pickled = pickle.dumps(datos)
    compressed = zlib.compress(pickled)
    return compressed

In [8]:
# Datos de las sesiones existentes
sesiones_info = [
    {"id": 1, "muestras": 5545},
    {"id": 2, "muestras": 18895}, 
    {"id": 3, "muestras": 15646}
]

In [10]:
print("🔄 Generando timestamps para las sesiones (comenzando en 0)...")
print("-" * 60)

for sesion in sesiones_info:
    session_id = sesion["id"]
    num_muestras = sesion["muestras"]
    
    # Generar el arreglo de timestamps comenzando en 0
    timestamps_ms = generar_timestamps_ms(num_muestras, 125)
    
    # Comprimir el arreglo
    datos_comprimidos = comprimir_datos(timestamps_ms)
    
    # Actualizar solo la columna datos_ms
    success = DatabaseManager.update_session(
        session_id=session_id,
        datos_ms=datos_comprimidos
    )
    
    if success:
        duracion_ms = timestamps_ms[-1]  # Último timestamp = duración total
        duracion_segundos = duracion_ms / 1000
        duracion_minutos = duracion_segundos / 60
        
        print(f"✅ Sesión {session_id}:")
        print(f"   - Muestras: {num_muestras:,}")
        print(f"   - Duración: {duracion_minutos:.2f} minutos ({duracion_segundos:.1f} segundos)")
        print(f"   - Primer timestamp: {timestamps_ms[0]} ms")
        print(f"   - Último timestamp: {timestamps_ms[-1]} ms")
        print(f"   - Intervalo: 8.0 ms entre muestras")
    else:
        print(f"❌ Error actualizando sesión {session_id}")

🔄 Generando timestamps para las sesiones (comenzando en 0)...
------------------------------------------------------------
✅ Sesión 1:
   - Muestras: 5,545
   - Duración: 0.74 minutos (44.4 segundos)
   - Primer timestamp: 0 ms
   - Último timestamp: 44352 ms
   - Intervalo: 8.0 ms entre muestras
✅ Sesión 2:
   - Muestras: 18,895
   - Duración: 2.52 minutos (151.2 segundos)
   - Primer timestamp: 0 ms
   - Último timestamp: 151152 ms
   - Intervalo: 8.0 ms entre muestras
✅ Sesión 3:
   - Muestras: 15,646
   - Duración: 2.09 minutos (125.2 segundos)
   - Primer timestamp: 0 ms
   - Último timestamp: 125160 ms
   - Intervalo: 8.0 ms entre muestras


In [11]:
print("🔍 Verificando datos guardados...")
print("-" * 60)

# Verificar que los datos se guardaron correctamente
for sesion in sesiones_info:
    session_id = sesion["id"]
    
    # Recuperar datos de la sesión
    session_data = DatabaseManager.get_session_data(session_id)
    
    if session_data and session_data.get("ms_data") is not None:
        ms_data = session_data["ms_data"]
        print(f"✅ Sesión {session_id} - Datos recuperados correctamente:")
        print(f"   - Número de timestamps: {len(ms_data):,}")
        print(f"   - Primer timestamp: {ms_data[0]} ms")
        print(f"   - Último timestamp: {ms_data[-1]} ms")
        print(f"   - Duración total: {ms_data[-1]/1000:.1f} segundos")
        print(f"   - Diferencia promedio: {np.mean(np.diff(ms_data)):.1f} ms")
    else:
        print(f"❌ Sesión {session_id} - Error recuperando datos")
    
    print()

print("📊 Resumen:")
print("-" * 60)
total_muestras = sum(s["muestras"] for s in sesiones_info)
print(f"Total de muestras procesadas: {total_muestras:,}")
print(f"Frecuencia de muestreo: 125 Hz")
print(f"Intervalo entre muestras: 8 ms")
print("Timestamps generados comenzando en 0 ms para cada sesión")

🔍 Verificando datos guardados...
------------------------------------------------------------
✅ Sesión 1 - Datos recuperados correctamente:
   - Número de timestamps: 5,545
   - Primer timestamp: 0 ms
   - Último timestamp: 44352 ms
   - Duración total: 44.4 segundos
   - Diferencia promedio: 8.0 ms

✅ Sesión 2 - Datos recuperados correctamente:
   - Número de timestamps: 18,895
   - Primer timestamp: 0 ms
   - Último timestamp: 151152 ms
   - Duración total: 151.2 segundos
   - Diferencia promedio: 8.0 ms

✅ Sesión 3 - Datos recuperados correctamente:
   - Número de timestamps: 15,646
   - Primer timestamp: 0 ms
   - Último timestamp: 125160 ms
   - Duración total: 125.2 segundos
   - Diferencia promedio: 8.0 ms

📊 Resumen:
------------------------------------------------------------
Total de muestras procesadas: 40,086
Frecuencia de muestreo: 125 Hz
Intervalo entre muestras: 8 ms
Timestamps generados comenzando en 0 ms para cada sesión
